In [1]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

## 2019 movies

In [4]:
PATH = './SplitTrailers_crop/'
trailers = os.listdir(PATH)
trailers.sort()
len(trailers)

97

## 2018 movies

In [8]:
PATH = './2018CropTrailers/'
df_onehot = pd.read_csv('./csv/filename2genreMat_2018.csv')
df_onehot

,id,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0328810,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,tt10005184,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt10017502,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,tt10043732,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,tt10048096,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,tt9866700,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1401,tt9879080,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1402,tt9891764,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1403,tt9904014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
trailers = df_onehot['id'].tolist()
trailers[:10]

['tt0328810',
 'tt10005184',
 'tt10017502',
 'tt10043732',
 'tt10048096',
 'tt10059624',
 'tt10077620',
 'tt10116528',
 'tt10160782',
 'tt10178206']

# A_AverageShotLength

In [8]:
A_li = []
for movie in trailers:
    frameCount = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' +scene + '/frames/'
        frameCount.append(len(os.listdir(framePath)))
    
#     print(frameCount)
    avg = np.mean(frameCount)
    std = np.std(frameCount)
    print(avg, std)
    
    A_li.append([avg, std])

Running: 47metersdown
34.80487804878049 23.842183882819985
Running: adogsjourneymovie
34.63247863247863 29.109753495301806
Running: aftermathmovie
35.69230769230769 26.611910622543594
Running: aftermovie
58.276595744680854 54.90405640467415
Running: alitamovie
35.78431372549019 29.192272497163273
Running: angelhasfallen
36.42156862745098 27.8171326366979
Running: angrybirdsmovie
34.65 30.91052948969547
Running: annabellemovie
52.36363636363637 45.788259830354015
Running: annamovie
37.91011235955056 33.61267833617541
Running: apollo11movie
65.6829268292683 58.86505403510322
Running: arcticmovie
72.625 94.80177863385717
Running: artofracingmovie
32.21212121212121 25.5755062460723
Running: avengers
63.70909090909091 71.98678482945772
Running: beachbummovie
33.64485981308411 20.565487983716434
Running: bernadettefilm
35.05555555555556 24.798384555111852
Running: blindedbythelightmovie
34.0990099009901 15.943902245739093
Running: booksmart
65.2 92.35521100620149
Running: breakthroughmovie
4

In [9]:
A_arr = np.asarray(A_li)
np.save('./npy/A_arr.npy', A_arr)
len(A_arr)

97

# B_ColorVariance
* https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_luv
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html

In [10]:
B_li = []
for movie in trailers: #order:
    rho = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        l = luv[:,:,0].ravel()
        u = luv[:,:,1].ravel()
        v = luv[:,:,2].ravel()
        x = np.asarray((l, u, v))
        cov = np.cov(x)
        det = np.linalg.det(cov)
        rho.append(det)
    
    avg = np.mean(rho)
    std = np.std(rho)
    print(avg, std)
    
    B_li.append([avg, std])

Running: 47metersdown
11937779.160579428 30622012.29551432
Running: adogsjourneymovie
2688809.223522822 4606428.453921334
Running: aftermathmovie
737843.0010296163 1447867.703815384
Running: aftermovie
5730162.1264876025 14299287.334890012
Running: alitamovie
2531454.7904955284 6295080.15899654
Running: angelhasfallen
1209756.4450619791 4667335.491197369
Running: angrybirdsmovie
369915511.07856596 751228830.3413738
Running: annabellemovie
4579331.774892684 34055753.876763575
Running: annamovie
9961093.345247298 30478166.89014458
Running: apollo11movie
6274530.625040619 15502910.006607102
Running: arcticmovie
419908.7930152336 1445080.8470512067
Running: artofracingmovie
10083699.589024222 22962107.516971376
Running: avengers
1709567.363935003 7479738.692924303
Running: beachbummovie
10557472.489571773 13171636.554763962
Running: bernadettefilm
8255672.661522896 12435050.248323254
Running: blindedbythelightmovie
9422088.193971574 46377098.979620636
Running: booksmart
3016540.562397561 4

In [11]:
B_arr = np.asarray(B_li)
np.save('./npy/B_arr.npy', B_arr)
len(B_arr)

97

# C_MotionContent
* https://blog.gtwang.org/programming/opencv-motion-detection-and-tracking-tutorial/

In [12]:
C_li = []
for movie in trailers: # [900:]:
    moving_li = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
    
        video  = PATH + movie + '/' + scene + '/clip.avi'

        cap = cv2.VideoCapture(video)

        w = int(cap.get(3))
        h = int(cap.get(4))

        # 計算畫面面積
        area = w * h

        # 初始化平均影像
        ret, frame = cap.read()

        avg = cv2.blur(frame,(4, 4))
        avg_float = np.float32(avg)
#         plt.imshow(frame)
#         plt.show()

        while(cap.isOpened()):
            # 讀取一幅影格
            ret, frame = cap.read()

            # 若讀取至影片結尾，則跳出
            if ret == False:
                break

            # 模糊處理
            blur = cv2.blur(frame, (4, 4))

            # 計算目前影格與平均影像的差異值
            diff = cv2.absdiff(frame, avg)

            # 將圖片轉為灰階
            gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # 篩選出變動程度大於門檻值的區域
            ret, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

            # 使用型態轉換函數去除雜訊
            kernel = np.ones((5, 5), np.uint8)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

            # 產生等高線
            cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            moving = 0
            for c in cnts:
                moving += cv2.contourArea(c)
                
            moving_li.append(moving/area)
            
#             for c in cnts:
#                 # 忽略太小的區域
#                 if cv2.contourArea(c) < 1000:
#                     continue


#             # 畫出等高線（除錯用）
#             cv2.drawContours(frame, cnts, -1, (0, 255, 255), 2)
#             plt.imshow(frame)
#             plt.show()

            # 更新平均影像
            cv2.accumulateWeighted(blur, avg_float, 0.01)
            avg = cv2.convertScaleAbs(avg_float)

        cap.release()
        cv2.destroyAllWindows()
    
    avg = np.mean(moving_li)
    std = np.std(moving_li)
    print(avg, std)
    
    C_li.append([avg, std])

Running: 47metersdown
0.24560857604689956 0.24974261700357553
Running: adogsjourneymovie
0.33599442968175286 0.349168693945698
Running: aftermathmovie
0.20429768996170855 0.2599513991686895
Running: aftermovie
0.212973970075358 0.2316550068940771
Running: alitamovie
0.2673263193152564 0.26963690207016533
Running: angelhasfallen
0.25070263657957015 0.28292585750320864
Running: angrybirdsmovie
0.3415629566501276 0.34148056657907155
Running: annabellemovie
0.09246866145816199 0.13234750118080257
Running: annamovie
0.3236956404202224 0.29021756294467305
Running: apollo11movie
0.18749989547658708 0.23665064533322383
Running: arcticmovie
0.2592962563654242 0.31775367317583825
Running: artofracingmovie
0.32656317688951997 0.34521141550011414
Running: avengers
0.2627411998693503 0.26814156391658595
Running: beachbummovie
0.19065556415466034 0.1999194087997236
Running: bernadettefilm
0.19538465196635715 0.2678772859709604
Running: blindedbythelightmovie
0.2198081136250925 0.2517577031617694
Run

In [13]:
C_arr = np.asarray(C_li)
np.save('./npy/C_arr.npy', C_arr)
len(C_arr)

97

# D_LightingKey

In [14]:
D_li = []
for movie in trailers:
    lk = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        value = hsv[:,:,2].ravel()
        mean = np.mean(value)
        std = np.std(value)
        lk.append(mean*std)

    avg = np.mean(lk)
    std = np.std(lk)
    print(avg, std)
    
    D_li.append([avg, std])

Running: 47metersdown
3832.149222456684 3285.912168339273
Running: adogsjourneymovie
3225.348524584389 2086.338315789782
Running: aftermathmovie
3212.997036429973 2613.398541739849
Running: aftermovie
3180.640515062546 2106.905406681398
Running: alitamovie
2845.366840575144 2350.3135919548176
Running: angelhasfallen
3686.115581034615 2869.0750183839014
Running: angrybirdsmovie
7726.730432469832 2614.3218269550307
Running: annabellemovie
1387.4944020693085 1821.0507450331909
Running: annamovie
4568.54899177322 2762.151807233616
Running: apollo11movie
4119.307187002707 2746.299971683731
Running: arcticmovie
2704.2718219421777 2309.4024877605807
Running: artofracingmovie
3901.6878242191683 2082.911548130717
Running: avengers
3105.39594032038 3241.181755533354
Running: beachbummovie
4604.061084524547 2391.8015119692454
Running: bernadettefilm
4734.0616987769545 1978.8730578672603
Running: blindedbythelightmovie
5058.690847604672 2780.5749886703898
Running: booksmart
3997.886217255943 2530.

In [15]:
D_arr = np.asarray(D_li)
np.save('./npy/D_arr.npy', D_arr)
len(D_arr)

97

# Results

In [16]:
# # 合併 2018 C_arr
# C_arr = np.append(np.load('./npy/2018C_arr_300.npy'), np.load('./npy/2018C_arr_600.npy'), axis=0)
# C_arr = np.append(C_arr, np.load('./npy/2018C_arr_900.npy'), axis=0)
# C_arr = np.append(C_arr, np.load('./npy/2018C_arr_900-end.npy'), axis=0)
# C_arr.shape

In [17]:
# reload computed values
A_arr = np.load('./npy/A_arr.npy')
B_arr = np.load('./npy/B_arr.npy')
C_arr = np.load('./npy/C_arr.npy')
D_arr = np.load('./npy/D_arr.npy')

(97, 2)

In [24]:
print(A_arr.shape, B_arr.shape, C_arr.shape, D_arr.shape)

(97, 2) (97, 2) (97, 2) (97, 2)


In [18]:
# First time succeed!
results = pd.DataFrame({
    'Movie': trailers,
    'A_avg': A_arr[:,0],
    'A_std': A_arr[:,1],
    'B_avg': B_arr[:,0],
    'B_std': B_arr[:,1],
    'C_avg': C_arr[:,0],
    'C_std': C_arr[:,1],
    'D_avg' : D_arr[:,0],
    'D_std' : D_arr[:,1]
})
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,47metersdown,34.804878,23.842184,1.193778e+07,3.062201e+07,0.245609,0.249743,3832.149222,3285.912168
1,adogsjourneymovie,34.632479,29.109753,2.688809e+06,4.606428e+06,0.335994,0.349169,3225.348525,2086.338316
2,aftermathmovie,35.692308,26.611911,7.378430e+05,1.447868e+06,0.204298,0.259951,3212.997036,2613.398542
3,aftermovie,58.276596,54.904056,5.730162e+06,1.429929e+07,0.212974,0.231655,3180.640515,2106.905407
4,alitamovie,35.784314,29.192272,2.531455e+06,6.295080e+06,0.267326,0.269637,2845.366841,2350.313592
...,...,...,...,...,...,...,...,...,...
92,usmovie,43.010204,52.851833,2.255390e+06,6.467894e+06,0.175808,0.220674,3444.948763,3236.149083
93,whatmenwant,32.086957,19.626320,2.003662e+07,5.276056e+07,0.241500,0.260970,6006.485656,2582.258884
94,wonderparkmovie,27.282443,13.193380,1.579958e+08,4.421705e+08,0.431886,0.331712,5554.107410,2757.092203
95,xmenmovies,41.549296,33.954729,4.791049e+06,1.487502e+07,0.319549,0.279627,2953.689532,2301.564008


## Normalize

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
def normalizing(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    print('Data range:', scaler.data_range_)
    return scaler.transform(data)

In [21]:
cols = results.columns[1:].tolist()
cols

['A_avg', 'A_std', 'B_avg', 'B_std', 'C_avg', 'C_std', 'D_avg', 'D_std']

In [22]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_norm'] = normalizing(arr)

Data range: [109.1367054]
Data range: [168.9261387]
Data range: [3.69648232e+08]
Data range: [7.50683084e+08]
Data range: [0.33941702]
Data range: [0.24881802]
Data range: [6527.36849728]
Data range: [2541.56075336]


In [23]:
results.to_csv('./csv/computed_2019.csv', index = 0)
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std,A_avg_norm,A_std_norm,B_avg_norm,B_std_norm,C_avg_norm,C_std_norm,D_avg_norm,D_std_norm
0,47metersdown,34.804878,23.842184,1.193778e+07,3.062201e+07,0.245609,0.249743,3832.149222,3285.912168,0.071101,0.066649,0.031572,0.040065,0.451185,0.471811,0.377956,0.831365
1,adogsjourneymovie,34.632479,29.109753,2.688809e+06,4.606428e+06,0.335994,0.349169,3225.348525,2086.338316,0.069522,0.097831,0.006551,0.005409,0.717482,0.871405,0.284994,0.359382
2,aftermathmovie,35.692308,26.611911,7.378430e+05,1.447868e+06,0.204298,0.259951,3212.997036,2613.398542,0.079233,0.083045,0.001273,0.001202,0.329474,0.512840,0.283102,0.566759
3,aftermovie,58.276596,54.904056,5.730162e+06,1.429929e+07,0.212974,0.231655,3180.640515,2106.905407,0.286168,0.250527,0.014779,0.018321,0.355036,0.399117,0.278145,0.367474
4,alitamovie,35.784314,29.192272,2.531455e+06,6.295080e+06,0.267326,0.269637,2845.366841,2350.313592,0.080076,0.098320,0.006125,0.007659,0.515171,0.551766,0.226780,0.463246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,usmovie,43.010204,52.851833,2.255390e+06,6.467894e+06,0.175808,0.220674,3444.948763,3236.149083,0.146285,0.238378,0.005378,0.007889,0.245535,0.354984,0.318637,0.811786
93,whatmenwant,32.086957,19.626320,2.003662e+07,5.276056e+07,0.241500,0.260970,6006.485656,2582.258884,0.046198,0.041692,0.053481,0.069556,0.439081,0.516934,0.711067,0.554507
94,wonderparkmovie,27.282443,13.193380,1.579958e+08,4.421705e+08,0.431886,0.331712,5554.107410,2757.092203,0.002175,0.003610,0.426699,0.588297,1.000000,0.801244,0.641762,0.623296
95,xmenmovies,41.549296,33.954729,4.791049e+06,1.487502e+07,0.319549,0.279627,2953.689532,2301.564008,0.132899,0.126512,0.012238,0.019088,0.669030,0.591916,0.243375,0.444065
